## Задача 5. Функция для фильтрации данных

#### Для оценки результатов экспериментов нам нужно выбрать данные, полученные во время эксперимента. Напишите функцию get_data_subset

In [40]:
from datetime import datetime

import pandas as pd


def get_data_subset(df, begin_date, end_date, user_ids=None, columns=None):
    """Возвращает подмножество данных.

    :param df (pd.DataFrame): таблица с данными, обязательные столбцы: 'date', 'user_id'.
    :param begin_date (datetime.datetime | None): дата начала интервала с данными.
        Пример, df[df['date'] >= begin_date].
        Если None, то фильтровать не нужно.
    :param end_date (datetime.datetime | None): дата окончания интервала с данными.
        Пример, df[df['date'] < end_date].
        Если None, то фильтровать не нужно.
    :param user_ids (list[str] | None): список user_id, по которым нужно предоставить данные.
        Пример, df[df['user_id'].isin(user_ids)].
        Если None, то фильтровать по user_id не нужно.
    :param columns (list[str] | None): список названий столбцов, по которым нужно предоставить данные.
        Пример, df[columns].
        Если None, то фильтровать по columns не нужно.

    :return df (pd.DataFrame): датафрейм с подмножеством данных.
    """
    result_df = df.copy(deep=True)

    if begin_date is not None:
        result_df = result_df[result_df['date'] >= begin_date]

    if end_date is not None:
        result_df = result_df[result_df['date'] < end_date]

    if user_ids is not None:
        result_df = result_df[result_df['user_id'].isin(user_ids)]

    if columns is not None:
        result_df = result_df[columns]

    return result_df

In [9]:
df = pd.DataFrame({
    'date': [datetime(2021, 10, 5), datetime(2022, 1, 5), datetime(2022, 1, 7)],
    'user_id': ['0', '1', '2'],
})

print(get_data_subset(df, datetime(2022, 1, 1), datetime(2022, 1, 6)))

        date user_id
1 2022-01-05       1


## Задача 6. Функции вычисления метрик

#### Напишите функции get_response_time, get_revenue_web и get_revenue_all для вычисления метрик «revenue (web)», «revenue (all)» и «response time»

In [1]:
from datetime import datetime
import pandas as pd

df_sales_, df_web_logs_ = pd.read_csv('sales.csv'), pd.read_csv('web_logs.csv')

In [50]:
import pandas as pd

from datetime import datetime


def get_response_time(df_web_logs, begin_date, end_date):
    """Вычисляет значения времени обработки запроса сервером.

    Нужно вернуть значения user_id и load_time из таблицы df_web_logs,
    отфильтрованные по дате.
    Считаем, что запросы обрабатываются независимо, поэтому группировать
    по user_id не нужно.

    :param df_web_logs (pd.DataFrame): таблица с логами сайта, содержит
    столбцы ['user_id', 'date', 'load_time'].
    :param begin_date, end_date (datetime): границы периода для
    фильтрации данных по дате. Левая граница входит, правая не входит.

    :return (pd.DataFrame): датафрейм с двумя столбцами ['user_id', 'metric']
    """
    result_df = df_web_logs[(df_web_logs['date'] >= begin_date) & (df_web_logs['date'] < end_date)][['user_id', 'load_time']]
    result_df = result_df.rename({'load_time': 'metric'}, axis=1)
    
    return result_df


def get_revenue_web(df_sales, df_web_logs, begin_date, end_date):
    """Вычисляет значения выручки с пользователя за указанный период
    для заходивших на сайт в указанный период.

    Эти данные нужны для экспериментов на сайте, когда в эксперимент
    попадают только те, кто заходил на сайт во время эксперимента.

    Нужно вернуть значения user_id и выручки (sum(price)) за указанный
    период для пользователей, заходивших на сайт в указанный период.
    Если пользователь зашёл на сайт и ничего не купил, его суммарная
    стоимость покупок равна нулю.
    Для каждого user_id должно быть ровно одно значение.

    :param df_sales (pd.DataFrame): таблица с продажами, содержит
        столбцы ['user_id', 'date', 'price'].
    :param df_web_logs (pd.DataFrame): таблица с логами сайта, содержит
        столбцы ['user_id', 'date', 'load_time'].
    :param begin_date, end_date (datetime): границы периода для фильтрации
        данных по дате. Левая граница входит, правая не входит.

    :return (pd.DataFrame): датафрейм с двумя столбцами ['user_id', 'metric']
    """
    
    df_web_logs['date']= pd.to_datetime(df_web_logs['date'])
    unique_visitors = df_web_logs[(df_web_logs['date'] >= begin_date) & (df_web_logs['date'] < end_date)][['user_id']]
    unique_visitors = unique_visitors.drop_duplicates()

    df_sales['date']= pd.to_datetime(df_sales['date'])
    df_sales = df_sales[(df_sales['date'] >= begin_date) & (df_sales['date'] < end_date)]

    result_df = unique_visitors.merge(df_sales, how='left', on='user_id')[['user_id', 'price']]
    result_df['price'] = result_df['price'].fillna(0)
    result_df = result_df.groupby('user_id')['price'].sum().reset_index()

    return result_df.rename({'price': 'metric'}, axis=1)


def get_revenue_all(df_sales, df_web_logs, begin_date, end_date):
    """Вычисляет значения выручки с пользователя за указанный период
    для заходивших на сайт до end_date.

    Эти данные нужны, например, для экспериментов с рассылкой по email,
    когда в эксперимент попадают те, кто когда-либо оставил нам свои данные.

    Нужно вернуть значения user_id и выручки (sum(price)) за указанный период
    для пользователей, заходивших на сайт до end_date.
    Если пользователь ничего не купил за указанный период, его суммарная
    стоимость покупок равна нулю.
    Для каждого user_id должно быть ровно одно значение.

    :param df_sales (pd.DataFrame): таблица с продажами, содержит
        столбцы ['user_id', 'date', 'price'].
    :param df_web_logs (pd.DataFrame): таблица с логами сайта, содержит
        столбцы ['user_id', 'date', 'load_time'].
    :param begin_date, end_date (datetime): границы периода для фильтрации
        данных по дате. Левая граница входит, правая не входит.

    :return (pd.DataFrame): датафрейм с двумя столбцами ['user_id', 'metric']
    """
    df_web_logs['date']= pd.to_datetime(df_web_logs['date'])
    unique_visitors = df_web_logs[df_web_logs['date'] < end_date][['user_id']]
    unique_visitors = unique_visitors.drop_duplicates()

    df_sales['date']= pd.to_datetime(df_sales['date'])
    df_sales = df_sales[(df_sales['date'] >= begin_date) & (df_sales['date'] < end_date)]

    result_df = unique_visitors.merge(df_sales, how='left', on='user_id')[['user_id', 'price']]
    result_df['price'] = result_df['price'].fillna(0)
    result_df = result_df.groupby('user_id')['price'].sum().reset_index().rename({'price': 'metric'}, axis=1)
    result_df['metric'] = result_df['metric'].astype(int)

    return result_df

In [51]:
df_sales = pd.DataFrame({
    'date': [datetime(2022, 3, day, 11) for day in range(11, 14)],
    'price': [1100, 900, 1500],
    'user_id': ['1', '2', '1'],
})
df_web_logs = pd.DataFrame({
    'date': [datetime(2022, 3, day, 11) for day in range(10, 14)],
    'load_time': [80.8, 90.1, 15.8, 19.7],
    'user_id': ['3', '1', '2', '1'],
})
begin_date = datetime(2022, 3, 11, 9)
end_date = datetime(2022, 4, 11, 9)

response_time = get_response_time(df_web_logs, begin_date, end_date)
# response_time = pd.DataFrame({'user_id': ['1', '2', '1'], 'metric': [90.1, 15.8, 19.7],})

revenue_web = get_revenue_web(df_sales, df_web_logs, begin_date, end_date)
# revenue_web = pd.DataFrame({'user_id': ['1', '2'], 'metric': [2600, 900],})

revenue_all = get_revenue_all(df_sales, df_web_logs, begin_date, end_date)
# revenue_all = pd.DataFrame({'user_id': ['1', '2', '3'], 'metric': [2600, 900, 0],})

In [52]:
print(response_time)

  user_id  metric
1       1    90.1
2       2    15.8
3       1    19.7


In [53]:
print(revenue_web)

  user_id  metric
0       1    2600
1       2     900


In [54]:
revenue_all

,user_id,metric
0,1,2600
1,2,900
2,3,0
